# Smart Grid 

In [31]:
import os
base_path = os.getcwd()
print(base_path)

/home/tim/Documents/work/apache_flink


In [ ]:
from pyflink.common import Configuration
from pyflink.table import TableEnvironment, EnvironmentSettings, DataTypes, TableDescriptor, Schema

# Set up the execution configuration
configuration = Configuration()
configuration.set_integer("table.exec.resource.default-parallelism", 1)

# Create the TableEnvironment in streaming mode
t_env = TableEnvironment.create(
    EnvironmentSettings.new_instance().in_streaming_mode().with_configuration(configuration).build()
)

# Create the input table for the smart-grid dataset
t_env.create_temporary_table(
    'smart_grid',
    TableDescriptor.for_connector('filesystem')
    .schema(
        Schema.new_builder()
            .column('creationTS', DataTypes.BIGINT())
            .column('test_value', DataTypes.FLOAT())
            .column('property', DataTypes.SMALLINT())
            .column('plug', DataTypes.SMALLINT())
            .column('household', DataTypes.SMALLINT())
            .column('house', DataTypes.SMALLINT())
            # Convert the epoch seconds to a TIMESTAMP_LTZ; multiply by 1000 to get milliseconds.
            .column_by_expression('eventTime', "TO_TIMESTAMP_LTZ(creationTS * 1000, 3)")
            .watermark('eventTime', "eventTime")
            .build()
    )
    .option('path', f'{base_path}/data/smart_grid/smartgrid-data.csv')
    .format('csv')
    .build()
)

# Create the sink table for Query 1
t_env.create_temporary_table(
    'sink_q1',
    TableDescriptor.for_connector('filesystem')
    .schema(
        Schema.new_builder()
            .column('globalAvgLoad', DataTypes.FLOAT())
            .column('window_start', DataTypes.BIGINT())
            .column('window_end', DataTypes.BIGINT())
            .build()
    )
    .option('path', f'{base_path}/data/smart_grid/outputs/sink_q1.csv')
    .format('csv')
    .build()
)

# Create the sink table for Query 2
t_env.create_temporary_table(
    'sink_q2',
    TableDescriptor.for_connector('filesystem')
    .schema(
        Schema.new_builder()
            .column('plug', DataTypes.SMALLINT())
            .column('household', DataTypes.SMALLINT())
            .column('house', DataTypes.SMALLINT())
            .column('localAvgLoad', DataTypes.FLOAT())
            .column('window_start', DataTypes.BIGINT())
            .column('window_end', DataTypes.BIGINT())
            .build()
    )
    .option('path', f'{base_path}/data/smart_grid/outputs/sink_q2.csv')
    .format('csv')
    .build()
)

# Create the sink table for Query 3
t_env.create_temporary_table(
    'sink_q3',
    TableDescriptor.for_connector('filesystem')
    .schema(
        Schema.new_builder()
            .column('plug', DataTypes.SMALLINT())
            .column('household', DataTypes.SMALLINT())
            .column('house', DataTypes.SMALLINT())
            .column('localAvgLoad', DataTypes.FLOAT())
            .column('window_start', DataTypes.BIGINT())
            .column('window_end', DataTypes.BIGINT())
            .build()
    )
    .option('path', f'{base_path}/data/smart_grid/outputs/sink_q3.csv')
    .format('csv')
    .build()
)

## Query1
```cpp
Query::from("sg")
    .window(SlidingWindow::of(EventTime(RecordCreationTs()), Seconds(3600), Seconds(1)))
    .apply(Avg(Attribute("value"))->as(Attribute("globalAvgLoad")))
    .sink(NullOutputSinkDescriptor::create());

In [37]:
t_env.execute_sql('''
INSERT INTO sink_q1
SELECT 
    AVG(test_value) AS globalAvgLoad,
    1000 * UNIX_TIMESTAMP(CAST(window_start AS STRING)) + EXTRACT(MILLISECOND FROM window_start) as `window_start`,
    1000 * UNIX_TIMESTAMP(CAST(window_end AS STRING)) + EXTRACT(MILLISECOND FROM window_end) as `window_end`
FROM TABLE(
    HOP(
         TABLE smart_grid,
         DESCRIPTOR(eventTime),
         INTERVAL '1' SECOND,   -- Slide interval
         INTERVAL '1' HOUR   -- Window size (1 hour)
    )
)
GROUP BY window_start, window_end;
''').wait()

## Query 2
```cpp
Query::from("sg")
    .window(SlidingWindow::of(EventTime(RecordCreationTs()), Seconds(3600), Seconds(1)))
    .byKey(Attribute("plug"), Attribute("household"), Attribute("house"))
    .apply(Avg(Attribute("value"))->as(Attribute("localAvgLoad")))
    .sink(NullOutputSinkDescriptor::create());

In [38]:
# Takes around 3min 40sec 
t_env.execute_sql('''
INSERT INTO sink_q2
SELECT 
    plug,
    household,
    house,
    AVG(test_value) AS localAvgLoad,
    1000 * UNIX_TIMESTAMP(CAST(window_start AS STRING)) + EXTRACT(MILLISECOND FROM window_start) AS window_start,
    1000 * UNIX_TIMESTAMP(CAST(window_end AS STRING)) + EXTRACT(MILLISECOND FROM window_end) AS window_end
FROM TABLE(
    HOP(
         TABLE smart_grid,
         DESCRIPTOR(eventTime),
         INTERVAL '1' SECOND,   -- Slide interval
         INTERVAL '1' HOUR      -- Window size (1 hour)
    )
)
GROUP BY 
    plug, 
    household, 
    house, 
    window_start, 
    window_end;
''').wait()

## Query 3
```cpp
Query::from("sg").window(SlidingWindow::of(EventTime(RecordCreationTs()), Seconds(128), Seconds(1)))
    .byKey(Attribute("plug"), Attribute("household"), Attribute("house"))
    .apply(Avg(Attribute("value"))->as(Attribute("localAvgLoad")))
    .sink(NullOutputSinkDescriptor::create());

In [36]:
t_env.execute_sql('''
INSERT INTO sink_q3
SELECT 
    plug,
    household,
    house,
    AVG(test_value) AS localAvgLoad,
    1000 * UNIX_TIMESTAMP(CAST(window_start AS STRING)) + EXTRACT(MILLISECOND FROM window_start) AS window_start,
    1000 * UNIX_TIMESTAMP(CAST(window_end AS STRING)) + EXTRACT(MILLISECOND FROM window_end) AS window_end
FROM TABLE(
    HOP(
         TABLE smart_grid,
         DESCRIPTOR(eventTime),
         INTERVAL '1' SECOND,   -- Slide interval
         INTERVAL '00:02:08' HOUR TO SECOND   -- Window size (128 seconds)
    )
)
GROUP BY 
    plug, 
    household, 
    house, 
    window_start, 
    window_end;
''').wait()

#### Script to convert data from txt to csv

In [2]:
import csv

# Define input and output file paths
input_file = f"{base_path}/data/smart_grid/smartgrid-data.txt"
output_file = f"{base_path}/data/smart_grid/smartgrid-data.csv"

with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for line in txt_file:
        # Split line by whitespace
        row = line.strip().split()
        # Write the row to CSV (all 6 columns)
        writer.writerow(row)